# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
from sklearn import preprocessing
import re
import string
from nltk.corpus import stopwords
import matplotlib.cm as cm
from matplotlib import rcParams
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('train.csv',delimiter=',',encoding='latin-1')

In [ ]:
data.head()

In [ ]:
data=data[['text','label']]

In [ ]:
data.head()

In [ ]:
print('Count of the dataset is: ', len(data))

#  Preprocessing

In [ ]:
data['text']=data['text'].str.lower()

In [ ]:
data['text'].head()

In [ ]:
data['label'] = data['label'].replace({0: 'FakeNews'})

In [ ]:
data['label'] = data['label'].replace({1: 'TrueNews'})

In [ ]:
data.columns

In [ ]:
data['label'].unique()

In [ ]:
data.dtypes

In [ ]:
data[data.isnull().any(axis=1)]

In [ ]:
import numpy as np
np.sum(data.isnull().any(axis=1))

In [ ]:
data=data.dropna()

In [ ]:
data[data.isnull().any(axis=1)]

In [ ]:
import numpy as np
np.sum(data.isnull().any(axis=1))

In [ ]:
data.info()

In [ ]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [ ]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
def processPost(text): 

    text = re.sub('@[^\s]+', ' ', text)
    

    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)

    text = re.sub(r'#([^\s]+)', r'\1', text)

    text= remove_punctuations(text)
    text=remove_repeating_char(text)
    
    return text

In [ ]:
data["text"] = data["text"].apply(lambda x: processPost(x))

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
data["text"] = data["text"].apply(tokenizer.tokenize)

In [ ]:
data["text"].head()

In [ ]:
stopwords_list = stopwords.words('english')

In [ ]:
stopwords_list

In [ ]:
data["text"]=data["text"].apply(lambda x: [item for item in x if item not in stopwords_list])

In [ ]:
data["text"].head()

In [ ]:
all_words = [word for tokens in data["text"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in data["text"]]

VOCAB = sorted(list(set(all_words)))

print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Maximum sentence length is %s" % max(sentence_lengths))

In [ ]:
counter = Counter(all_words)

In [ ]:
counter.most_common(25)

In [ ]:
counted_words = Counter(all_words)

words = []
counts = []
for letter, count in counted_words.most_common(25):
    words.append(letter)
    counts.append(count)

In [ ]:
colors = cm.rainbow(np.linspace(0, 1, 10))
rcParams['figure.figsize'] = 20, 10

plt.title('Top words in Tweet_Text')
plt.xlabel('Count')
plt.ylabel('Words')
plt.barh(words, counts, color=colors)

# Features Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features =1500)

In [ ]:
unigramdataGet= word_vectorizer.fit_transform(data['text'].astype('str'))
unigramdataGet = unigramdataGet.toarray()
vocab = word_vectorizer.get_feature_names()
Descripcion_features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
Descripcion_features[Descripcion_features>0] = 1

In [ ]:
data.reset_index(drop=True, inplace=True)
data=data.drop(columns=['text'])

In [ ]:
data = pd.concat([Descripcion_features, data['label']], axis=1)

In [ ]:
data.to_csv('procecessed_data.csv', index=False)